## 初始化通用参数

In [ ]:
import argparse
import os
import sys
sys.argv = ['run.py']

from mmengine import Config, DictAction
from mmengine.config import Config
from mmpose.utils import register_all_modules
from mmengine.hooks import Hook
from mmengine.runner import Runner

import initialize


#命名规则 {{算法信息}}_{{模块信息}}_{{训练信息}}_{{数据信息}}
CONFIG='../configs/animal_2d_keypoint/topdown_heatmap/animalpose/td-hm_res101_8xb64-210e_animalpose-256x256.py'
DATASET='V1_985'
REMARK='NoAug'
NAME='{}_{}_{}'.format(CONFIG.split('/')[-1].split('.')[0],DATASET,REMARK)

parser = argparse.ArgumentParser()
parser.add_argument('--config',default=CONFIG,help='配置文件')
parser.add_argument('--work-dir',default=f'./results/{NAME}/',type=str,help='工作区根目录')
parser.add_argument('--data-root',default=f'./datasets/{DATASET}',type=str,help='数据集根目录')
parser.add_argument(
        '--cfg-options',
        nargs='+',
        action=DictAction,
        help='覆盖配置，键值对 '
        'in xxx=yyy format will be merged into config file. If the value to '
        'be overwritten is a list, it should be like key="[a,b]" or key=a,b '
        'It also allows nested list/tuple values, e.g. key="[(a,b),(c,d)]" '
        'Note that the quotation marks are necessary and that no white space '
        'is allowed.')
args = parser.parse_args()

#自定义配置文件
cfg=initialize.get_cfg(args) 
cfg.dump('config_tmp.py')


## 展示数据

In [ ]:
parser.add_argument('--output-dir',default=args.work_dir+'dataset_show/',type=str,help='输出根目录')
parser.add_argument('--not_show', default=True)
parser.add_argument('--phase',default='val',type=str,choices=['train', 'test', 'val'],help='查看某阶段的数据集')
parser.add_argument('--show_interval',type=float,default=2,help='显示的间隔')
parser.add_argument('--mode',default='transformed',type=str,choices=['original', 'transformed'],
    help='展示原图或增强效果'
    'pictures. "original" means to show images load from disk'
    '; "transformed" means to show images after transformed;'
    'Defaults to "transformed".')
args = parser.parse_args()

%run ../tools/misc/browse_dataset.py config_tmp.py --output-dir {args.output_dir} --phase {args.phase} --mode {args.mode}

## 训练

In [ ]:
parser.add_argument('--resume',nargs='?',type=str,const='auto',
    help='If specify checkpint path, resume from it, while if not '
    'specify, try to auto resume from the latest checkpoint '
    'in the work directory.')
parser.add_argument('--amp', action='store_true',default=False,help='启用自动混合精度训练')
parser.add_argument('--no-validate',action='store_true',help='是否在训练中验证')
parser.add_argument('--auto-scale-lr',action='store_true',help='是否自动缩放学习率')
parser.add_argument('--show-dir',default=args.work_dir+'train_show/',help='保存可视化图像的目录')
parser.add_argument('--show',action='store_true', help='是否直接展示预测结果')
parser.add_argument('--interval',type=int,default=1,help='visualize per interval samples.')
parser.add_argument('--wait-time',type=float,default=1,help='display time of every window. (second)')
parser.add_argument('--launcher',choices=['none', 'pytorch', 'slurm', 'mpi'],default='none',help='job launcher')
parser.add_argument('--local_rank', type=int, default=0)
args = parser.parse_args()
if 'LOCAL_RANK' not in os.environ:
    os.environ['LOCAL_RANK'] = str(args.local_rank)

%run ../tools/train.py config_tmp.py  --show-dir {args.show_dir}


## 测试

In [ ]:
parser.add_argument('--checkpoint',default=r"E:\Desktop\pig_hoof\results\td-hm_res101_8xb64-210e_animalpose-256x256_V985_NoAug\epoch_210.pth", help='checkpoint file')
# parser.add_argument('--work-dir',default=args.work_dir+'eval/', help='the directory to save evaluation results')
parser.add_argument('--out',default=args.work_dir+'eval/eval.txt', help='保存结果的文件')
parser.add_argument('--dump',type=str,default=args.work_dir+'eval/eval.pkl',help='将预测转储到pickle文件以进行离线评估')
parser.add_argument('--show-dir',default=args.work_dir+'eval/show/',help='保存测试图像的推理结果')
parser.add_argument('--show',default=True,help='是否直接展示测试结果')
parser.add_argument('--interval',type=int,default=1,help='visualize per interval samples.')
parser.add_argument('--wait-time',type=float,default=1,help='display time of every window. (second)')
parser.add_argument('--launcher',choices=['none', 'pytorch', 'slurm', 'mpi'],default='none',help='job launcher')
parser.add_argument('--local_rank', type=int, default=0)
args.work_dir=args.work_dir+'eval/'
args = parser.parse_args()


%run ../tools/test.py config_tmp.py {args.checkpoint} --out {args.out} --show-dir {args.show_dir} --work-dir {args.work_dir}
